# Etude Snorkel / Extration de partenariats d'entreprises
Pour ce projet, nous allons essayer d'extraire des relations de partenariats économiques entre des entreprises depuis des articles. <br/>
Le but étant de comprendre l'utilisation de snorkel.

Nous allons découper notre traitement dans 3 notebooks (chacun correspondant à une étape du pipeline `Snorkel` :
1. Preprocessing
2. Training
3. Evaluation

## Partie III: Entrainement d'un modèle d'extraction final

Dans ce notebook, nous allons utiliser nos labels (bruités) que nous avons générés dans le notebook précédent pour entrainer notre modèle d'extraction final.

Nous allons entrainer un Bi-LSTM qui fait parti de l'état de l'art des réseaux d'apprentissage profond.

***

In [64]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from snorkel.models import candidate_subclass
from snorkel.annotations import load_marginals
from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession

#from snorkel.learning.pytorch import LSTM
#from snorkel.learning.pytorch.rnn import LSTM

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# I. Chargement des données

On initialise notre schéma.

In [76]:
Partner = candidate_subclass('Partner', ['company1', 'company2'])

Chargement des labels d'entrainement probabilistes (_marginals_)

In [83]:
train_marginals = load_marginals(session, split=0)

Chargement des candidats

In [84]:
train_cands = session.query(Partner).filter(Partner.split == 0).all()
dev_cands   = session.query(Partner).filter(Partner.split == 1).all()
test_cands  = session.query(Partner).filter(Partner.split == 2).all()

Chargement des gold labels pour l'évaluation de notre modélisation.

In [85]:
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

# II. Entrainement d'un modèle LSTM d'extraction

In [90]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':              0.01,
    'embedding_dim':   100,
    'hidden_dim':      100,
    'n_epochs':        12,
    'dropout':         0.5,
    'rebalance':       0.25,
    'print_freq':      1,
    'seed':            1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=1764  #epochs=12  batch size=64
[LSTM] Epoch 1 (13.10s)	Average loss=0.595623	Dev F1=41.46
[LSTM] Epoch 2 (24.37s)	Average loss=0.529658	Dev F1=49.02
[LSTM] Epoch 3 (36.14s)	Average loss=0.522368	Dev F1=49.44
[LSTM] Epoch 4 (52.08s)	Average loss=0.517288	Dev F1=47.31
[LSTM] Epoch 5 (70.22s)	Average loss=0.512169	Dev F1=50.00
[LSTM] Epoch 6 (82.74s)	Average loss=0.508329	Dev F1=50.00
[LSTM] Epoch 7 (95.93s)	Average loss=0.507193	Dev F1=43.59
[LSTM] Epoch 8 (111.31s)	Average loss=0.506622	Dev F1=48.35
[LSTM] Epoch 9 (123.31s)	Average loss=0.505472	Dev F1=49.44
[LSTM] Epoch 10 (141.09s)	Average loss=0.504679	Dev F1=48.28
[LSTM] Epoch 11 (155.21s)	Average loss=0.504336	Dev F1=49.46
[LSTM] Model saved as <LSTM>
[LSTM] Epoch 12 (170.48s)	Average loss=0.503958	Dev F1=45.24
[LSTM] Training done (170.77s)
[LSTM] Loaded model <LSTM>


## Score sur jeu de développement

In [88]:
p, r, f1 = lstm.score(dev_cands, L_gold_dev)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.478, Recall: 0.512, F1 Score: 0.494


## Score sur Test

In [87]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.536, Recall: 0.407, F1 Score: 0.463
